# Rating predictions - Piotr Falkiewicz

+ The phase of data preprocessing was skipped due to unsatifsying results (no improvement of results).
+ Solution consists of 2 models: Linear SVR (regression) and Linear SVC (classification).
+ In the last stage, results of both are mixed to improve final prediction.

**1. Import of modules**

In [37]:
import pandas as pd
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

**2. Data preparation and NaN's handling**

In [4]:
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/test.csv")

test_nan = test_data[test_data['Reviews'].isna()]
test_nan['Rating'] = 5
test_data = test_data.dropna(subset=['Reviews'])
train_data = train_data.dropna(subset=['Reviews'])

value = train_data['Rating'].tolist()
features = train_data['Reviews'].tolist()
features2 = test_data['Reviews'].tolist()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


**3. TF-IDF features preparation**

In [5]:
vectorizer = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,3)) 

vectors = vectorizer.fit_transform(features+features2)

**4. Linear SVC**

In [7]:
classifier = svm.LinearSVC()
 
classifier.fit(vectors[:len(features)], value)

predictionsC = classifier.predict(vectors[len(features):])

**5. Linear SVR**

In [8]:
classifier = svm.LinearSVR(loss='squared_epsilon_insensitive')
 
classifier.fit(vectors[:len(features)], value)

predictionsR = classifier.predict(vectors[len(features):])

**6. Predictions merge**

In [35]:
test_data['RatingR'] = predictionsR
test_data['RatingC'] = predictionsC
test_data['Rating'] = test_data.apply(lambda x: int(round(x['RatingR'])) if abs(int(round(x['RatingR'])) - x['RatingC']) > 1 and int(round(x['RatingR'])) > 0 else x['RatingC'], axis = 1)

**7. Export to file**

In [36]:
predictions = test_data.loc[:,['Id','Rating']]
predictions_nan = test_nan.loc[:,['Id','Rating']]
final_predictions = pd.concat([predictions, predictions_nan])
final_predictions.sort_values(by='Id').to_csv('results_jupter.csv', index = False)